In [ ]:
!pip install wfdb
import wfdb
import matplotlib.pyplot as plt
import wfdb.processing
import pandas as pd
!pip install matplotlib==3.1.3
import numpy as np 


name_column = 's0010_re'
q_onset_column = 1340
t_end_column = 1756
df = pd.read_excel('/content/qt.xlsx')

In [ ]:
list_of_ecg = [name_column]
previous_patient = 'patient001'
for index, name in enumerate(df[name_column]):
  if df.patient1[index] == 'patient285':
    continue
  print(previous_patient)
  if not pd.isna(df.patient1[index]):
    num = str(df.patient1[index][7:])
    if len(num) == 1:
      num = f'00{num}'
    elif len(num) == 2:
      num = f'0{num}'
    previous_patient = f'{df.patient1[index][:-int(len(str(df.patient1[index][7:])))]}{num}'
  list_of_ecg.append(wfdb.rdrecord(name, sampto=int(df[t_end_column][index]) + 5000, 
                                     pn_dir = f'ptbdb/{previous_patient}',
                                     channels=[0]))

In [11]:
list_of_ecg[0] = wfdb.rdrecord(list_of_ecg[0], sampto=int(t_end_column) + 5000, 
                                     pn_dir = 'ptbdb/patient001',
                                     channels=[0])

In [13]:
list_of_q_onset = df[q_onset_column]
q_onset = [int(q_onset_column)]
list_of_t_end = df[t_end_column]
t_end = [int(t_end_column)]
for i in range(len(list_of_q_onset)):
  if list_of_q_onset[i] == 0:
    continue
  else:
    q_onset.append(list_of_q_onset[i])
    t_end.append(list_of_t_end[i])

In [ ]:
!pip install scipy
from scipy import signal
from scipy.signal import butter, iirnotch, lfilter
from scipy.signal import butter, sosfilt, sosfilt_zi, sosfiltfilt, lfilter, lfilter_zi, filtfilt, sosfreqz, resample

In [15]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    sos = butter(order, [low, high], analog=False, btype="band", output="sos")
    return sos


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfilt(sos,
                data)  
    return y


def butter_bandpass_filter_once(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    zi = sosfilt_zi(sos)
    z, _ = sosfilt(sos, data, zi=zi * data[0])
    return sos, z, zi


def butter_bandpass_filter_again(sos, z, zi):
    z2, _ = sosfilt(sos, z, zi=zi * z[0])
    return z2


def butter_bandpass_forward_backward_filter(data, lowcut, highcut, fs, order=5):
    sos = butter_bandpass(lowcut, highcut, fs, order=order)
    y = sosfiltfilt(sos,data, axis = 0) 
    return y

In [16]:
lowcut = 1 
highcut = 20
copy_ecg = list_of_ecg.copy()
for i in range(len(copy_ecg)):
  fs = list_of_ecg[i].fs
  copy_ecg[i] = butter_bandpass_forward_backward_filter(copy_ecg[i].p_signal, lowcut, highcut, fs, order=4)

In [ ]:
k = 23
min_bpm = 20
max_bpm = 230
search_radius = int(60 * list_of_ecg[k].fs / max_bpm)
qrs_inds = wfdb.processing.qrs.gqrs_detect(sig=copy_ecg[k], fs=list_of_ecg[k].fs)
peaks = wfdb.processing.correct_peaks(copy_ecg[k].flatten(), peak_inds=qrs_inds,
                    search_radius=search_radius,
                    smooth_window_size = 150, peak_dir='up')
left_slice = 0
right_slice = 0
fig, ax = plt.subplots()
ax.plot(copy_ecg[k])
for j in peaks:
  ax.axvline(x=j, color='g')
for indexes in range(len(peaks)):
  if peaks[indexes] > q_onset[k]:
    if indexes == 0:
      left_slice = 0
    else:
      left_slice = peaks[indexes - 1]
    right_slice = peaks[indexes + 1]
    break

In [ ]:
sliced_ecg = copy_ecg.copy()
ecg_peak = []
ecg = []
q = []
t = []
for i in range(len(sliced_ecg)):
  min_bpm = 20
  max_bpm = 230
  search_radius = int(60 * list_of_ecg[i].fs / max_bpm)
  qrs_inds = wfdb.processing.qrs.gqrs_detect(sig=sliced_ecg[i], fs=list_of_ecg[i].fs)
  peaks = wfdb.processing.correct_peaks(sliced_ecg[i].flatten(), peak_inds=qrs_inds,
                      search_radius=search_radius,
                     smooth_window_size = 150, peak_dir='up')
  left_slice = 0
  right_slice = 0
  if len(peaks) == 0 or peaks[0] > t_end[i] + 1000:
    continue
  p = 0
  for indexes in range(len(peaks)):
    if peaks[indexes] > q_onset[i] and len(peaks) > 1:
      if indexes == 0:
        left_slice = 0
      else:
        left_slice = peaks[indexes - 1]
      right_slice = peaks[indexes + 1]
      p = indexes
      break
  ecg.append(sliced_ecg[i][left_slice:right_slice])
  ecg_peak.append(p)
  q.append(q_onset[i] - left_slice)
  t.append(t_end[i] - left_slice)
fig, ax = plt.subplots()

In [20]:
final_ecg = []
final_q = []
peak = []
final_t = []
for i,x in enumerate(ecg):
  if len(x) > 800:
    final_ecg.append(x)
    final_q.append(q[i])
    peak.append(ecg_peak[i])
    final_t.append(t[i])

In [22]:
min_ecg_length = 800
lst_of_ecg = []
q = []
t = [] 
p = []
for i in range(len(final_ecg)):
  ecg = final_ecg[i]
  distance = len(ecg) - min_ecg_length
  if distance % 3 == 0:
    left = (distance // 3) * 2 
    right = len(ecg) - (distance // 3)
  elif distance % 3 == 1:
    left = (distance // 3) * 2
    right = len(ecg) - (distance // 3) - 1
  else:
    left = (distance // 3) * 2
    right = len(ecg) - (distance // 3) - 2
  if left > peak[i] and peak[i] > 0:
    if left < 50:
      right -= left
      left = 0 
    else:
      left -= 50
      right -= 50
  if (final_q[i] - left < 0):
    continue
  lst_of_ecg.append(ecg[left:right])
  q.append(final_q[i] - left)
  t.append(final_t[i] - left)
  p.append(peak[i] - left)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
for i in range(len(lst_of_ecg)):
  lst_of_ecg[i] = lst_of_ecg[i].flatten()
train_x, test_x, train_y, test_y = train_test_split(lst_of_ecg, q, test_size=0.05, random_state=42)
model = GradientBoostingRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model.fit(train_x, train_y)
y_pred = model.predict(test_x)
mean_absolute_error(test_y, y_pred)

In [ ]:
train_x_t, test_x_t, train_y_t, test_y_t = train_test_split(lst_of_ecg, t, test_size=0.05, random_state=42)
model_t = GradientBoostingRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model_t.fit(train_x_t, train_y_t)
y_pred_t = model_t.predict(test_x_t)
mean_absolute_error(test_y_t, y_pred_t)

In [ ]:
# measurements of metrics for different models for Q onset
model = Ridge(100, random_state=42)
model1 = Lasso(1, random_state=42)
model2 = GradientBoostingRegressor(max_depth=5, criterion="absolute_error", random_state=42)
model3 = GradientBoostingRegressor(max_depth=4, criterion="absolute_error", random_state=42)
model4 = GradientBoostingRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model5 = GradientBoostingRegressor(max_depth=2, criterion="absolute_error", random_state=42)
model6 = RandomForestRegressor(max_depth=5, criterion="absolute_error", random_state=42)
model7 = RandomForestRegressor(max_depth=4, criterion="absolute_error", random_state=42)
model8 = RandomForestRegressor(max_depth=3, criterion="absolute_error", random_state=42)
model9 = RandomForestRegressor(max_depth=2, criterion="absolute_error", random_state=42)
Mmodel2 = GradientBoostingRegressor(max_depth=5,  random_state=42)
Mmodel3 = GradientBoostingRegressor(max_depth=4,  random_state=42)
Mmodel4 = GradientBoostingRegressor(max_depth=3,  random_state=42)
Mmodel5 = GradientBoostingRegressor(max_depth=2,  random_state=42)
Mmodel6 = RandomForestRegressor(max_depth=5,  random_state=42)
Mmodel7 = RandomForestRegressor(max_depth=4,  random_state=42)
Mmodel8 = RandomForestRegressor(max_depth=3, random_state=42)
Mmodel9 = RandomForestRegressor(max_depth=2, random_state=42)
lst_of_models = [model, model1, model2, model3, model4, model5, model6, model7, model8, model9,
                Mmodel2, Mmodel3, Mmodel4, Mmodel5, Mmodel6, Mmodel7, Mmodel8, Mmodel9]
lst_of_deviation = [0.] * 18
for x, i in enumerate(lst_of_models):
  i.fit(train_x, train_y)
  pr = i.predict(test_x)
  lst_of_deviation[x] = mean_absolute_error(test_y, pr)

In [ ]:
lst_of_titles = [ 
                 "Ridge(100)",
                 "Lasso(1)",
                 "GradientBoostingRegressor(max_depth=5, absolute error)",
                 "GradientBoostingRegressor(max_depth=4, absolute error)",
                 "GradientBoostingRegressor(max_depth=3, absolute error)",
                 "GradientBoostingRegressor(max_depth=2, absolute error)",
                 "RandomForestRegressor(max_depth=5, absolute error)",
                 "RandomForestRegressor(max_depth=4, absolute error)",
                 "RandomForestRegressor(max_depth=3, absolute error)",
                 "RandomForestRegressor(max_depth=2, absolute error)"
                 "GradientBoostingRegressor(max_depth=5)",
                 "GradientBoostingRegressor(max_depth=4)",
                 "GradientBoostingRegressor(max_depth=3)",
                 "GradientBoostingRegressor(max_depth=2)",
                 "RandomForestRegressor(max_depth=5)",
                 "RandomForestRegressor(max_depth=4)",
                 "RandomForestRegressor(max_depth=3)",
                 "RandomForestRegressor(max_depth=2)"
]

In [ ]:
for i in range(len(lst_of_titles)):
  print(f"{lst_of_titles[i]} MAE: {lst_of_deviation[i]}")

In [ ]:
interval_length = [0.] * len(y_pred)
annotated_interval_length = [0.] * len(y_pred)
for i in range(len(interval_length)):
  interval_length[i] = y_pred_t[i] - y_pred[i]
  annotated_interval_length[i] = test_y_t[i] - test_y[i]
mean_absolute_error(interval_length, annotated_interval_length)

In [ ]:
# Check deviation of test dataset
import statistics
import math 
dev = statistics.pvariance(test_y_t)
dev = math.sqrt(dev)
dev

In [ ]:
dev = statistics.pvariance(test_y)
dev = math.sqrt(dev)
dev

In [ ]:
# measurements of metrics for different models for T onset
lst_of_deviation = [0.] * 18
for x, i in enumerate(lst_of_models):
  i.fit(train_x_t, train_y_t)
  pr = i.predict(test_x_t)
  lst_of_deviation[x] = mean_absolute_error(test_y_t, pr)

In [ ]:
for i in range(len(lst_of_titles)):
  print(f"{lst_of_titles[i]} MAE: {lst_of_deviation[i]}")